# Using SEC EDGAR RESTful data APIs

This notebook shows how to retrieve information reported by regulated entities to U.S. Securities and Exchange Commision (SEC).

SEC is maintainig EDGAR system with information about all regulated enties (companies, funds, individuals). Accessing the data is free and there is number of [various ways how to access the data](https://www.sec.gov/os/accessing-edgar-data).

"data.sec.gov" was created to host RESTful data Application Programming Interfaces (APIs) delivering JSON-formatted data to external customers and to web pages on SEC.gov. These APIs do not require any authentication or API keys to access.

Currently included in the APIs are the submissions history by filer and the XBRL data from financial statements (forms 10-Q, 10-K,8-K, 20-F, 40-F, 6-K, and their variants).

The JSON structures are updated throughout the day, in real time, as submissions are disseminated.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# data are published in JSON format so we will need json library
import json


# Finding CIK of company

EDGAR assigns to filers a unique numerical identifier, known as a Central Index Key (CIK), when they sign up to make filings to the SEC. CIK numbers remain unique to the filer; they are not recycled. 

List of all CIKs matched with entity name is available for download [(13 MB, text file)](https://www.sec.gov/Archives/edgar/cik-lookup-data.txt). Note that this list includes funds and individuals and is historically cumulative for company names. Thus a given CIK may be associated with multiple names in the case of company or fund name changes, and the list contains some entities that no longer file with the SEC.

We will be using smaller (611 kB) JSON [kaggle dataset](https://www.kaggle.com/datasets/svendaj/sec-edgar-cik-ticker-exchange), which is sourcing data directly at EDGAR and is input for this notebook. This dataset contains only companies names, CIK, ticker and associated stock exchange.

In [7]:
# Let's convert CIK JSON to pandas DataFrame
# First load the data into python dictionary

with open("company_tickers_exchange.json", "r") as f:
    CIK_dict = json.load(f)



In [8]:
# dataset contains two sections
CIK_dict.keys()

dict_keys(['fields', 'data'])

In [9]:
# fields is specifying meaning and and order of company data
# we will use it as columns names
CIK_dict["fields"]

['cik', 'name', 'ticker', 'exchange']

In [10]:
# data section is list of records/lists for each company
# we will use it as DataFrame rows
print("Number of company records:", len(CIK_dict["data"]))
CIK_dict["data"][:5]    # first 5 records

Number of company records: 10379


[[789019, 'MICROSOFT CORP', 'MSFT', 'Nasdaq'],
 [320193, 'Apple Inc.', 'AAPL', 'Nasdaq'],
 [1045810, 'NVIDIA CORP', 'NVDA', 'Nasdaq'],
 [1652044, 'Alphabet Inc.', 'GOOGL', 'Nasdaq'],
 [1018724, 'AMAZON COM INC', 'AMZN', 'Nasdaq']]

In [11]:
# convert CIK_dict to pandas
CIK_df = pd.DataFrame(CIK_dict["data"], columns=CIK_dict["fields"])
CIK_df

,cik,name,ticker,exchange
0,789019,MICROSOFT CORP,MSFT,Nasdaq
1,320193,Apple Inc.,AAPL,Nasdaq
2,1045810,NVIDIA CORP,NVDA,Nasdaq
3,1652044,Alphabet Inc.,GOOGL,Nasdaq
4,1018724,AMAZON COM INC,AMZN,Nasdaq
...,...,...,...,...
10374,1995413,COLOMBIER ACQUISITION CORP. II,CLBR-WT,NYSE
10375,1998781,IB Acquisition Corp.,IBACU,Nasdaq
10376,2000775,Black Hawk Acquisition Corp,BKHAU,Nasdaq
10377,2002038,Legato Merger Corp. III,LEGT-UN,NYSE


## Select the ticker of company used in this example

Subsequent information retrieval will be using selected `ticker` and associated CIK

In [12]:
# finding company row with given ticker
ticker = "AMMX"
CIK_df[CIK_df["ticker"] == ticker]

,cik,name,ticker,exchange
8419,1776048,Ameramex International Inc,AMMX,OTC


In [13]:
CIK = CIK_df[CIK_df["ticker"] == ticker].cik.values[0]

In [14]:
# finding companies containing substring in company name
substring = "oil"
CIK_df[CIK_df["name"].str.contains(substring, case=False)]

,cik,name,ticker,exchange
358,49938,IMPERIAL OIL LTD,IMO,NYSE
668,101778,MARATHON OIL CORP,MRO,NYSE
897,1327068,"United States Oil Fund, LP",USO,NYSE
1075,717423,MURPHY OIL CORP,MUR,NYSE
1277,1698990,Magnolia Oil & Gas Corp,MGY,NYSE
1412,1104485,"NORTHERN OIL & GAS, INC.",NOG,NYSE
3186,74046,Oil-Dri Corp of America,ODC,NYSE
3301,1383058,Invesco DB Oil Fund,DBO,NYSE
3483,1697500,"Solaris Oilfield Infrastructure, Inc.",SOI,NYSE
3545,1121484,"OIL STATES INTERNATIONAL, INC",OIS,NYSE


# Entity’s current filing history

Each entity’s current filing history is available at the following URL:

* https://data.sec.gov/submissions/CIK##########.json

Where the ########## is the entity’s 10-digit Central Index Key (CIK), including leading zeros.

This JSON data structure contains metadata such as current name, former name, and stock exchanges and ticker symbols of publicly-traded companies. The object’s property path contains at least one year’s of filing or to 1,000 (whichever is more) of the most recent filings in a compact columnar data array. If the entity has additional filings, files will contain an array of additional JSON files and the date range for the filings each one contains.

In [20]:
# preparation of input data, using ticker and CIK set earlier
CIK="320193"
url = f"https://data.sec.gov/submissions/CIK{str(CIK).zfill(10)}.json"
url

'https://data.sec.gov/submissions/CIK0000320193.json'

# Reading from RESTful API

EDGAR requires that HTTP requests will be identified with proper [UserAgent in header and comply with fair use policy (currently max. 10 requests per second)](https://www.sec.gov/os/accessing-edgar-data). At minimum you need to supply your own e-mail adress in User-Agent field (otherwise you will get 403/Forbiden error). If you will provide Host field, please be sure use data.sec.gov server and not www.sec.gov as mentioned in example (this would result in 404/Not Found error).

In [21]:
# read response from REST API with `requests` library and format it as python dict

import requests
header = {
  "User-Agent": "your.email@email.com"#, # remaining fields are optional
#    "Accept-Encoding": "gzip, deflate",
#    "Host": "data.sec.gov"
}

company_filings = requests.get(url, headers=header).json()
company_filings.keys()

dict_keys(['cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'])

In [22]:
company_filings["addresses"]

{'mailing': {'street1': 'ONE APPLE PARK WAY',
  'street2': None,
  'city': 'CUPERTINO',
  'stateOrCountry': 'CA',
  'zipCode': '95014',
  'stateOrCountryDescription': 'CA'},
 'business': {'street1': 'ONE APPLE PARK WAY',
  'street2': None,
  'city': 'CUPERTINO',
  'stateOrCountry': 'CA',
  'zipCode': '95014',
  'stateOrCountryDescription': 'CA'}}

In [23]:
company_filings["filings"]["recent"].keys()

dict_keys(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'])

# Creating DataFrame with submitted filings

`company_filings["filings"]["recent"]` contains up to 1000 last submitted filings sorted from latest to oldest.

In [24]:
company_filings_df = pd.DataFrame(company_filings["filings"]["recent"])
company_filings_df

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000320193-24-000058,2024-04-17,2024-04-15,2024-04-17T18:30:52.000Z,,4,,,,16177,0,0,xslF345X05/wk-form4_1713393040.xml,FORM 4
1,0000320193-24-000056,2024-04-15,2024-04-11,2024-04-15T18:31:11.000Z,,4,,,,5376,0,0,xslF345X05/wk-form4_1713220262.xml,FORM 4
2,0000320193-24-000055,2024-04-15,2024-04-11,2024-04-15T18:30:24.000Z,,4,,,,8967,0,0,xslF345X05/wk-form4_1713220215.xml,FORM 4
3,0001958244-24-001666,2024-04-11,,2024-04-11T16:40:51.000Z,33,144,001-36743,24838992,,5403,0,0,xsl144X01/primary_doc.xml,
4,0001973141-24-000220,2024-04-11,,2024-04-11T16:10:08.000Z,33,144,001-36743,24838608,,4414,0,0,xsl144X01/primary_doc.xml,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0001181431-13-064577,2013-12-26,2013-12-23,2013-12-26T18:30:47.000Z,,4,,,,8519,0,0,xslF345X03/rrd397758.xml,13.12.23 RICCIO FORM 4 - RSU VESTING
996,0001181431-13-064133,2013-12-20,2013-12-17,2013-12-20T18:36:33.000Z,,4,,,,4981,0,0,xslF345X03/rrd398053.xml,13.12.17 LEVINSON FORM 4 - GIFT TRANSFER
997,9999999997-14-000040,2013-12-17,2013-10-18,2014-01-08T11:47:13.000Z,34,NO ACT,000-10030,13003879,,1381,0,0,9999999997-14-000040.paper,AUTO-GENERATED PAPER DOCUMENT
998,0000928464-13-000275,2013-12-04,,2013-12-04T15:37:31.000Z,34,DFAN14A,000-10030,131257242,,33501,0,0,aapldfan14a120413.htm,


In [16]:
# filter only Annual reports
company_filings_df[company_filings_df.form == "10-K"]

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
6,0001099910-22-000055,2022-03-31,2021-12-31,2022-03-31T16:42:41.000Z,34,10-K,000-56054,22793428,,4555112,1,1,ammx_10k.htm,ANNUAL REPORT
13,0001099910-21-000051,2021-04-16,2020-12-31,2021-04-15T17:32:07.000Z,34,10-K,000-56054,21829489,,4294621,1,0,ammx_10k.htm,ANNUAL REPORT
30,0001099910-20-000042,2020-06-05,2019-12-31,2020-06-05T16:30:22.000Z,34,10-K,000-56054,20946702,,3879921,1,0,ammx_10k.htm,ANNUAL REPORT


# Accessing specific filing document

Let's download latest Annual Report (10-K). Files are stored in browsable directory structure for CIK and accession-number: 
* https://www.sec.gov/Archives/edgar/data/{CIK}/{accession-number}/

In [28]:
access_number = company_filings_df[company_filings_df.form == "10-K"].accessionNumber.values[0].replace("-", "")

file_name = company_filings_df[company_filings_df.form == "10-K"].primaryDocument.values[0]

url = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{access_number}/{file_name}"
url

'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm'

In [29]:
# dowloading and saving requested document to working directory
req_content = requests.get(url, headers=header).content.decode("utf-8")

with open(file_name, "w") as f:
    f.write(req_content)

## and saving it as PDF


In [19]:
pip install weasyprint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.5/704.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 57.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
from weasyprint import HTML

HTML(string=req_content, base_url="").write_pdf(file_name + ".pdf")

In [21]:
!ls -al .

total 1340
drwxr-xr-x 2 root root    4096 Feb 18 12:39 .
drwxr-xr-x 6 root root    4096 Feb 18 12:38 ..
---------- 1 root root  116359 Feb 18 12:39 __notebook__.ipynb
-rw-r--r-- 1 root root 1002883 Feb 18 12:38 ammx_10k.htm
-rw-r--r-- 1 root root  239246 Feb 18 12:39 ammx_10k.htm.pdf


# XBRL data APIs

Extensible Business Markup Language (XBRL) is an XML-based format for reporting financial statements used by the SEC and financial regulatory agencies across the world. XBRL, in a separate XML file or more recently embedded in quarterly and annual HTML reports as inline XBRL, was first required by the SEC in 2009. XBRL facts must be associated for a standard US-GAAP or IFRS taxonomy. Companies can also extend standard taxonomies with their own custom taxonomies.

The following XBRL APIs aggregate facts from across submissions that
1. Use a non-custom taxonomy (e.g. us-gaap, ifrs-full, dei, or srt)
1. Apply to the entire filing entity

This ensures that facts have a consistent context and meaning across companies and between filings and are comparable between companies and across time.

## All company concepts data
## data.sec.gov/api/xbrl/companyfacts/

This API returns all the company concepts data for a company into a single API call:

* https://data.sec.gov/api/xbrl/companyfacts/CIK##########.json

In [22]:
url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{str(CIK).zfill(10)}.json"
url

'https://data.sec.gov/api/xbrl/companyfacts/CIK0001776048.json'

In [23]:
company_facts = requests.get(url, headers=header).json()

# get the current assets values as reported over time and make it pandas DataFrame
curr_assets_df = pd.DataFrame(company_facts["facts"]["us-gaap"]["AssetsCurrent"]["units"]["USD"])
curr_assets_df

,end,val,accn,fy,fp,form,filed,frame
0,2018-12-31,3808259,0001099910-19-000059,2019,Q1,10-Q/A,2019-07-18,NaN
1,2018-12-31,3808259,0001099910-19-000073,2019,Q2,10-Q,2019-08-14,NaN
2,2018-12-31,3808259,0001099910-19-000095,2019,Q3,10-Q,2019-11-14,NaN
3,2018-12-31,3808259,0001099910-20-000042,2019,FY,10-K,2020-06-05,CY2018Q4I
4,2019-03-31,5464979,0001099910-19-000059,2019,Q1,10-Q/A,2019-07-18,CY2019Q1I
5,2019-06-30,4383839,0001099910-19-000073,2019,Q2,10-Q,2019-08-14,CY2019Q2I
6,2019-09-30,6114050,0001099910-19-000095,2019,Q3,10-Q,2019-11-14,CY2019Q3I
7,2019-12-31,5743442,0001099910-20-000042,2019,FY,10-K,2020-06-05,NaN
8,2019-12-31,5743442,0001099910-20-000058,2020,Q1,10-Q,2020-06-24,NaN
9,2019-12-31,5743442,0001099910-20-000079,2020,Q2,10-Q,2020-08-14,NaN


In [24]:
# get just values reported in valid frame and plot them
curr_assets_df[curr_assets_df.frame.notna()]


,end,val,accn,fy,fp,form,filed,frame
3,2018-12-31,3808259,0001099910-20-000042,2019,FY,10-K,2020-06-05,CY2018Q4I
4,2019-03-31,5464979,0001099910-19-000059,2019,Q1,10-Q/A,2019-07-18,CY2019Q1I
5,2019-06-30,4383839,0001099910-19-000073,2019,Q2,10-Q,2019-08-14,CY2019Q2I
6,2019-09-30,6114050,0001099910-19-000095,2019,Q3,10-Q,2019-11-14,CY2019Q3I
11,2019-12-31,5743442,0001099910-21-000051,2020,FY,10-K,2021-04-16,CY2019Q4I
12,2020-03-31,7657896,0001099910-20-000058,2020,Q1,10-Q,2020-06-24,CY2020Q1I
13,2020-06-30,8805973,0001099910-20-000079,2020,Q2,10-Q,2020-08-14,CY2020Q2I
14,2020-09-30,9675545,0001099910-20-000108,2020,Q3,10-Q,2020-11-13,CY2020Q3I
19,2020-12-31,7248352,0001099910-22-000055,2021,FY,10-K,2022-03-31,CY2020Q4I
20,2021-03-31,8963989,0001099910-21-000059,2021,Q1,10-Q,2021-05-17,CY2021Q1I


In [25]:
import plotly.express as px
pd.options.plotting.backend = "plotly" 

curr_assets_df.plot(x="end", y="val", 
                    title=f"{company_filings['name']}, {ticker}: Current Assets",
                   labels= {
                       "val": "Value ($)",
                       "end": "Quarter End"
                   })

## Getting datapoints of single concept
## data.sec.gov/api/xbrl/companyconcept/

The company-concept API returns all the XBRL disclosures from a single company (CIK) and concept (a taxonomy and tag) into a single JSON file, with a separate array of facts for each units on measure that the company has chosen to disclose (e.g. net profits reported in U.S. dollars and in Canadian dollars).

* https://data.sec.gov/api/xbrl/companyconcept/CIK##########/us-gaap/AccountsPayableCurrent.json


In [26]:
# let's retrieve current assets for comparision with company facts API
url = f"https://data.sec.gov/api/xbrl/companyconcept/CIK{str(CIK).zfill(10)}/us-gaap/AssetsCurrent.json"
url

'https://data.sec.gov/api/xbrl/companyconcept/CIK0001776048/us-gaap/AssetsCurrent.json'

In [27]:
curr_assets_dict = requests.get(url, headers=header).json()
curr_assets_dict.keys()

dict_keys(['cik', 'taxonomy', 'tag', 'label', 'description', 'entityName', 'units'])

In [28]:
curr_assets_dict["tag"]

'AssetsCurrent'

In [29]:
# first 5 datapoints
curr_assets_dict["units"]["USD"][:5]

[{'end': '2018-12-31',
  'val': 3808259,
  'accn': '0001099910-19-000059',
  'fy': 2019,
  'fp': 'Q1',
  'form': '10-Q/A',
  'filed': '2019-07-18'},
 {'end': '2018-12-31',
  'val': 3808259,
  'accn': '0001099910-19-000073',
  'fy': 2019,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2019-08-14'},
 {'end': '2018-12-31',
  'val': 3808259,
  'accn': '0001099910-19-000095',
  'fy': 2019,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2019-11-14'},
 {'end': '2018-12-31',
  'val': 3808259,
  'accn': '0001099910-20-000042',
  'fy': 2019,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2020-06-05',
  'frame': 'CY2018Q4I'},
 {'end': '2019-03-31',
  'val': 5464979,
  'accn': '0001099910-19-000059',
  'fy': 2019,
  'fp': 'Q1',
  'form': '10-Q/A',
  'filed': '2019-07-18',
  'frame': 'CY2019Q1I'}]

In [30]:
# this should be resulting in same DataFrame as retrieved through companyfacts API and selected through taxonomy us-gaap, AssetsCurrent concept/tag and units USD
curr_assets_df = pd.DataFrame(curr_assets_dict["units"]["USD"])
curr_assets_df

,end,val,accn,fy,fp,form,filed,frame
0,2018-12-31,3808259,0001099910-19-000059,2019,Q1,10-Q/A,2019-07-18,NaN
1,2018-12-31,3808259,0001099910-19-000073,2019,Q2,10-Q,2019-08-14,NaN
2,2018-12-31,3808259,0001099910-19-000095,2019,Q3,10-Q,2019-11-14,NaN
3,2018-12-31,3808259,0001099910-20-000042,2019,FY,10-K,2020-06-05,CY2018Q4I
4,2019-03-31,5464979,0001099910-19-000059,2019,Q1,10-Q/A,2019-07-18,CY2019Q1I
5,2019-06-30,4383839,0001099910-19-000073,2019,Q2,10-Q,2019-08-14,CY2019Q2I
6,2019-09-30,6114050,0001099910-19-000095,2019,Q3,10-Q,2019-11-14,CY2019Q3I
7,2019-12-31,5743442,0001099910-20-000042,2019,FY,10-K,2020-06-05,NaN
8,2019-12-31,5743442,0001099910-20-000058,2020,Q1,10-Q,2020-06-24,NaN
9,2019-12-31,5743442,0001099910-20-000079,2020,Q2,10-Q,2020-08-14,NaN


## Getting one fact from requested period/frame
## data.sec.gov/api/xbrl/frames/

The xbrl/frames API aggregates one fact for **each** reporting entity that is last filed that most closely fits the calendrical period requested. This API supports for annual, quarterly and instantaneous data:

* https://data.sec.gov/api/xbrl/frames/us-gaap/AccountsPayableCurrent/USD/CY2019Q1I.json

Where the units of measure specified in the XBRL contains a numerator and a denominator, these are separated by “-per-” such as “USD-per-shares”. Note that the default unit in XBRL is “pure”.

The period format is CY#### for annual data (duration 365 days +/- 30 days), CY####Q# for quarterly data (duration 91 days +/- 30 days), and CY####Q#I for instantaneous data. Because company financial calendars can start and end on any month or day and even change in length from quarter to quarter to according to the day of the week, the frame data is assembled by the dates that best align with a calendar quarter or year. Data users should be mindful different reporting start and end dates for facts contained in a frame.

In [31]:
# Let's retrieve all data about current assets in Q4 of 2021
fact = "AssetsCurrent"
year = 2021
quarter = "Q1I"

url = f"https://data.sec.gov/api/xbrl/frames/us-gaap/{fact}/USD/CY{year}{quarter}.json"
url

'https://data.sec.gov/api/xbrl/frames/us-gaap/AssetsCurrent/USD/CY2021Q1I.json'

In [32]:
curr_assets_dict = requests.get(url, headers=header).json()
curr_assets_dict.keys()

dict_keys(['taxonomy', 'tag', 'ccp', 'uom', 'label', 'description', 'pts', 'data'])

In [33]:
# let's convert all data of requested period to pandas dataframe
curr_assets_df = pd.DataFrame(curr_assets_dict["data"])
curr_assets_df.sort_values("val", ascending=False)

,accn,cik,entityName,loc,end,val
3624,0001652044-21-000020,1652044,Alphabet Inc.,US-CA,2021-03-31,172137000000
705,0001564590-21-020891,789019,Microsoft Corporation,US-WA,2021-03-31,165614000000
1848,0001156375-21-000052,1156375,CME GROUP INC.,US-IL,2021-03-31,126745500000
486,0000320193-21-000056,320193,Apple Inc.,US-CA,2021-03-27,121465000000
1349,0001018724-21-000010,1018724,"AMAZON.COM, INC.",US-WA,2021-03-31,121408000000
...,...,...,...,...,...,...
4419,0001096906-21-001190,1811999,"FARMHOUSE, INC. /NV",US-CA,2021-03-31,0
3568,0001829126-21-010477,1642365,BALINCAN INTERNATIONAL INC.,US-NY,2021-03-31,0
2674,0001731122-21-000886,1445175,"Xiamen Lutong International Travel Agency Co.,...",-,2021-03-31,0
2038,0001477932-21-003750,1284454,Yong Bai Chao New Retail Corp,-,2021-03-31,0


In [34]:
company_facts["facts"].keys()

dict_keys(['dei', 'us-gaap'])

In [35]:
company_facts["entityName"]

'AMERAMEX INTERNATIONAL, INC.'

In [36]:
CIK = 320193
url = f"https://data.sec.gov/api/xbrl/companyconcept/CIK{str(CIK).zfill(10)}/dei/EntityRegistrantName.json"
url

'https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/dei/EntityRegistrantName.json'

![SEC logo](https://www.pngkit.com/png/detail/177-1773725_seal-of-the-united-states-securities-and-exchange.png)